# 

In [1]:
from googleapiclient.discovery import build 
import os
import pandas as pd
from IPython.display import JSON

In [2]:
# reading api_key from file
api_key = ""
with open("key.txt","r") as f:
    api_key = f.read()

In [3]:
    # Get credentials and create an API client

api_service_name = "youtube"
api_version = "v3"
youtube = build (
    api_service_name, api_version, developerKey=api_key )
channel_id = "UCX6OQ3DkcsbYNE6H8uQQuVA"

In [4]:
# -*- coding: utf-8 -*-
channel_id = "UCX6OQ3DkcsbYNE6H8uQQuVA"
def get_channels(youtube,channel_id):
    total_data = []
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=channel_id
    )
    response = request.execute()

    # loop through items
    for item in response['items']:
        data = {
                "Channel_Name" : item['snippet']['title'],
                "Suscribers" : item['statistics']['subscriberCount'],
                "View" : item['statistics']['viewCount'],
                "TotalVideo" : item['statistics']['videoCount'],
                "PlaylistId" : item['contentDetails']['relatedPlaylists']['uploads']
               }

    total_data.append(data)
    return pd.DataFrame(total_data)

def get_videos(youtube, video_id):
    video_ids = []
    request = youtube.playlistItems().list(
        part="snippet,contentDetails",
        playlistId=video_id,
        maxResults = 50
    )
    response = request.execute()
    
    for item in response['items']:
        video_ids.append(item['contentDetails']['videoId'])
    next_page_token = response.get('nextPageToken')

    while next_page_token is not None:
        request = youtube.playlistItems().list(
            part="snippet,contentDetails",
            playlistId=video_id,
            maxResults = 50,
            pageToken=next_page_token
        )
        response = request.execute()
        for item in response['items']:
            video_ids.append(item['contentDetails']['videoId'])
        next_page_token = response.get('nextPageToken')
        # print(len(video_ids))
        
    return video_ids

channel_data = get_channels(youtube, channel_id)
video_id = str(channel_data['PlaylistId'][0]).strip()
video_ids =  get_videos(youtube, video_id)
# print(video_ids[0])


# 

In [5]:
request = youtube.videos().list(
        part="snippet,contentDetails,statistics",
        id = video_ids[0:49]
    )
response = request.execute()
JSON(response)

<IPython.core.display.JSON object>

In [11]:
#create a dictionary which will store columns of our data
my_video_data = {
    'video_id' : [],
    'channel_title' : [],
    'title' : [],
    'description' : [],
    'published_at' : [],
    'view_count' : [],
    'like_Count' : [],
    'favourite_Count' : [],
    'duration' : [],
    'caption' : [],
    'defination' : [],
    'commentCount' : []
}


### ->inserting data in dataset

In [ ]:
for item in response['items']:
    my_video_data['video_id'].append(item['id'])
    my_video_data['channel_title'].append(item['snippet']['channelTitle'])
    my_video_data['title'].append(item['snippet']['title'])
    my_video_data['description'].append(item['snippet']['description'])
    my_video_data['like_Count'].append(item['statistics']['likeCount'])
    my_video_data['favourite_Count'].append(item['statistics']['favoriteCount'])
    my_video_data['duration'].append(item['contentDetails']['duration'])
    my_video_data['caption'].append(item['contentDetails']['caption'])
    my_video_data['defination'].append(item['contentDetails']['definition'])
    my_video_data['view_count'].append(item['statistics']['viewCount'])
    my_video_data['published_at'].append(item['snippet']['publishedAt'])
    my_video_data['commentCount'].append(item['statistics']['commentCount'])


### ->Displaying data

In [ ]:
df = pd.DataFrame(my_video_data)
df.head()

In [16]:
#checking for null values
df.isnull().any()

video_id           False
channel_title      False
title              False
description        False
published_at       False
view_count         False
like_Count         False
favourite_Count    False
duration           False
caption            False
defination         False
commentCount       False
dtype: bool